## Categorizing news articles according to their sentiment using POS tagging with a PMI-IR classifier. Such an approach eliminates the need for labelled training data.



In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import math
print("imports complete")

imports complete


### Importing dataframe- used Postman to access the NYT API for all newspaper headlines for the month of January 2020

In [2]:
df = pd.read_json("jan_2020.json")
df_2 = pd.DataFrame(df['response']['docs'])
article_df = df_2
article_df

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,...,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name,slideshow_credits
0,"Attacks have been traditionally underreported,...",https://www.nytimes.com/2020/01/03/us/anti-sem...,"Attacks have been traditionally underreported,...",When slurs were spray-painted on a kosher mark...,A,16,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': '‘A Different Era’: Anti-Semitic Crim...,"[{'name': 'subject', 'value': 'ANTI-SEMITISM',...",...,article,National,U.S.,"{'original': 'By Adeel Hassan', 'person': [{'f...",News,nyt://article/ab477781-8dc9-5ef0-b800-62057d30...,1015,nyt://article/ab477781-8dc9-5ef0-b800-62057d30...,NaN,NaN
1,He deftly manipulated electronic images to pro...,https://www.nytimes.com/2020/01/01/arts/woody-...,He deftly manipulated electronic images to pro...,"Woody Vasulka, an experimental video artist wh...",A,21,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Woody Vasulka, Whose Video Art Exten...","[{'name': 'persons', 'value': 'Vasulka, Woody'...",...,article,Obits,Arts,"{'original': 'By Richard Sandomir', 'person': ...",Obituary (Obit),nyt://article/2df26ba3-d0f2-5b77-8bef-3fc61954...,1015,nyt://article/2df26ba3-d0f2-5b77-8bef-3fc61954...,NaN,NaN
2,The past definitely wasn’t past for directors ...,https://www.nytimes.com/2020/01/02/movies/osca...,The past definitely wasn’t past for directors ...,"It’s Oscar time, which means everything new is...",AR,14,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Will the Oscars Get as Nostalgic as ...,"[{'name': 'subject', 'value': 'Movies', 'rank'...",...,article,Arts&Leisure,Movies,{'original': 'By Manohla Dargis and A.O. Scott...,News,nyt://article/b496c678-15f2-5f3a-8d12-66bcfe73...,2076,nyt://article/b496c678-15f2-5f3a-8d12-66bcfe73...,NaN,NaN
3,Forget fad diets and fitness gimmicks. Just st...,https://www.nytimes.com/2020/01/03/opinion/sim...,Forget fad diets and fitness gimmicks. Just st...,"Daytime television talk shows, popular podcast...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'How to Be Healthy, in Just 48 Words'...","[{'name': 'subject', 'value': 'Diet and Nutrit...",...,article,OpEd,Opinion,"{'original': 'By Yoni Freedhoff', 'person': [{...",Op-Ed,nyt://article/3664dd8e-fb90-5862-8ea3-4d5c409c...,1145,nyt://article/3664dd8e-fb90-5862-8ea3-4d5c409c...,NaN,NaN
4,Linda Tracy and Peter Tracy had been married 3...,https://www.nytimes.com/2020/01/03/fashion/wed...,Linda Tracy and Peter Tracy had been married 3...,When Donna Adelsberg ran into her best friend’...,ST,10,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'I Do. Take 2.', 'kicker': 'vows', 'c...","[{'name': 'subject', 'value': 'Divorce, Separa...",...,article,Styles,Fashion & Style,"{'original': 'By Tammy La Gorce', 'person': [{...",News,nyt://article/21594211-daf1-5b77-897e-38022da1...,1958,nyt://article/21594211-daf1-5b77-897e-38022da1...,Weddings,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7580,The city could become one of the first in the ...,https://www.nytimes.com/2020/01/30/nyregion/su...,The city could become one of the first in the ...,New York City should consider allowing yellow ...,A,22,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'New York Is Urged to Consider Surge ...,"[{'name': 'subject', 'value': 'Taxicabs and Ta...",...,article,Metro,New York,"{'original': 'By Brian M. Rosenthal', 'person'...",News,nyt://article/9ff7cae3-3afa-5b7d-8c81-2ab26344...,1150,nyt://article/9ff7cae3-3afa-5b7d-8c81-2ab26344...,NaN,NaN
7581,A New York City commission’s recommendations f...,https://www.nytimes.com/2020/01/31/nyregion/pr...,A New York City commission’s recommendations f...,"On a stretch of 11th Street i

### Data preprocessing

In [3]:
# Sentiments associated with different categories

article_df_cols = article_df.columns
for col_name in article_df_cols:
    print(col_name,":",article_df[col_name][25],"\n")

abstract : A meeting with President Volodymyr Zelensky of Ukraine would have been the first for a top Trump official since the impeachment inquiry began. 

web_url : https://www.nytimes.com/2020/01/01/us/politics/pompeo-cancels-ukraine-trip.html 

snippet : A meeting with President Volodymyr Zelensky of Ukraine would have been the first for a top Trump official since the impeachment inquiry began. 

lead_paragraph : WASHINGTON — Secretary of State Mike Pompeo on Wednesday canceled a weeklong trip to Ukraine and four other nations to stay in Washington and monitor tensions in Iraq after protesters broke into the United States Embassy compound in Baghdad and wrecked parts of it, the State Department said. 

print_section : A 

print_page : 9 

source : The New York Times 

multimedia : [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2020/01/01/us/politics/01DC-DIPLO/merlin_166477125_e8e576ff-b755-414c-b168-490b0746d5ed-articleLarge.jpg',

In [4]:
# we don't need some columns
article_df.drop(columns = ['slideshow_credits', 'subsection_name', 'uri', '_id', 'pub_date','multimedia', 'source','print_page','web_url', 'document_type','byline','section_name','word_count','type_of_material'], inplace = True, errors = 'ignore')
article_df.head()

,abstract,snippet,lead_paragraph,print_section,headline,keywords,news_desk
0,"Attacks have been traditionally underreported,...","Attacks have been traditionally underreported,...",When slurs were spray-painted on a kosher mark...,A,{'main': '‘A Different Era’: Anti-Semitic Crim...,"[{'name': 'subject', 'value': 'ANTI-SEMITISM',...",National
1,He deftly manipulated electronic images to pro...,He deftly manipulated electronic images to pro...,"Woody Vasulka, an experimental video artist wh...",A,"{'main': 'Woody Vasulka, Whose Video Art Exten...","[{'name': 'persons', 'value': 'Vasulka, Woody'...",Obits
2,The past definitely wasn’t past for directors ...,The past definitely wasn’t past for directors ...,"It’s Oscar time, which means everything new is...",AR,{'main': 'Will the Oscars Get as Nostalgic as ...,"[{'name': 'subject', 'value': 'Movies', 'rank'...",Arts&Leisure
3,Forget fad diets and fitness gimmicks. Just st...,Forget fad diets and fitness gimmicks. Just st...,"Daytime television talk shows, popular podcast...",NaN,"{'main': 'How to Be Healthy, in Just 48 Words'...","[{'name': 'subject', 'value': 'Diet and Nutrit...",OpEd
4,Linda Tracy and Peter Tracy had been married 3...,Linda Tracy and Peter Tracy had been married 3...,When Donna Adelsberg ran into her best friend’...,ST,"{'main': 'I Do. Take 2.', 'kicker': 'vows', 'c...","[{'name': 'subject', 'value': 'Divorce, Separa...",Styles


In [5]:
#streamline keywords
keywords = article_df['keywords']
new_kw = [[word['value'] for word in kw] for kw in keywords]
article_df['keywords'] = new_kw
article_df

,abstract,snippet,lead_paragraph,print_section,headline,keywords,news_desk
0,"Attacks have been traditionally underreported,...","Attacks have been traditionally underreported,...",When slurs were spray-painted on a kosher mark...,A,{'main': '‘A Different Era’: Anti-Semitic Crim...,"[ANTI-SEMITISM, Hate Crimes, New York City, Ch...",National
1,He deftly manipulated electronic images to pro...,He deftly manipulated electronic images to pro...,"Woody Vasulka, an experimental video artist wh...",A,"{'main': 'Woody Vasulka, Whose Video Art Exten...","[Vasulka, Woody, Deaths (Obituaries), Art, Kit...",Obits
2,The past definitely wasn’t past for directors ...,The past definitely wasn’t past for directors ...,"It’s Oscar time, which means everything new is...",AR,{'main': 'Will the Oscars Get as Nostalgic as ...,"[Movies, ACADEMY AWARDS (OSCARS), 1917 (Movie)...",Arts&Leisure
3,Forget fad diets and fitness gimmicks. Just st...,Forget fad diets and fitness gimmicks. Just st...,"Daytime television talk shows, popular podcast...",NaN,"{'main': 'How to Be Healthy, in Just 48 Words'...","[Diet and Nutrition, Exercise, Weight, Cooking...",OpEd
4,Linda Tracy and Peter Tracy had been married 3...,Linda Tracy and Peter Tracy had been married 3...,When Donna Adelsberg ran into her best friend’...,ST,"{'main': 'I Do. Take 2.', 'kicker': 'vows', 'c...","[Divorce, Separations and Annulments, Baltimor...",Styles
...,...,...,...,...,...,...,...
7580,The city could become one of the first in the ...,The city could become one of the first in the ...,New York City should consider allowing yellow ...,A,{'main': 'New York Is Urged to Consider Surge ...,"[Taxicabs and Taxicab Drivers, Uber Technologi...",Metro
7581,A New York City commission’s recommendations f...,A New York City commission’s recommendations f...,"On a stretch of 11th Street in Park Slope, Bro...",A,{'main': 'Battle Lines Quickly Form Over Radic...,"[Property Taxes, Real Estate and Housing (Resi...",Metro
7582,Seth Meyers took a closer look at Dershowitz’s...,Seth Meyers took a closer look at Dershowitz’s...,"Welcome to Best of Late Night, a rundown of th...",NaN,{'main': 'Late Night Dissects Alan Dershowitz’...,"[Bolton, John R, Colbert, Stephen, Dershowitz,...",Culture
7583,The secretary of state met with President Volo...,The secretary of state met with President Volo...,"KYIV, Ukraine — Secretary of State Mike Pompeo...",A,"{'main': 'Amid Trump’s Trial, Pompeo Visited U...",[Trump-Ukraine Whistle-blower Complaint and Im...,Foreign


In [6]:
#streamline headline

headlines = article_df['headline']
new_headlines = [headline['print_headline'] for headline in headlines]
new_headlines
article_df['headline'] = new_headlines

In [7]:
article_df.head(10)

,abstract,snippet,lead_paragraph,print_section,headline,keywords,news_desk
0,"Attacks have been traditionally underreported,...","Attacks have been traditionally underreported,...",When slurs were spray-painted on a kosher mark...,A,Scrutiny Reveals a Rise in Anti-Semitic Crimes,"[ANTI-SEMITISM, Hate Crimes, New York City, Ch...",National
1,He deftly manipulated electronic images to pro...,He deftly manipulated electronic images to pro...,"Woody Vasulka, an experimental video artist wh...",A,"Woody Vasulka, Who Foresaw Video as Art, Dies ...","[Vasulka, Woody, Deaths (Obituaries), Art, Kit...",Obits
2,The past definitely wasn’t past for directors ...,The past definitely wasn’t past for directors ...,"It’s Oscar time, which means everything new is...",AR,The Pros and Cons of Nostalgia,"[Movies, ACADEMY AWARDS (OSCARS), 1917 (Movie)...",Arts&Leisure
3,Forget fad diets and fitness gimmicks. Just st...,Forget fad diets and fitness gimmicks. Just st...,"Daytime television talk shows, popular podcast...",NaN,None,"[Diet and Nutrition, Exercise, Weight, Cooking...",OpEd
4,Linda Tracy and Peter Tracy had been married 3...,Linda Tracy and Peter Tracy had been married 3...,When Donna Adelsberg ran into her best friend’...,ST,"A Love So Persistent, They Had to Marry Twice","[Divorce, Separations and Annulments, Baltimor...",Styles
5,Split categories allow voters to spread the we...,Split categories allow voters to spread the we...,What will happen at the Golden Globes? Part of...,C,A Crystal Ball For an Event Prone to Jolts,"[Movies, Golden Globes (Awards), Hustlers (Mov...",Culture
6,E.S.P. Conjuring the dead. Speaking with them....,E.S.P. Conjuring the dead. Speaking with them....,"Listen. Just listen. Not with your ears, but w...",C,"Séance and Theater, Making Believers","[Theater, Waters, Les, White, Whitney, Coward,...",Culture
7,"Juwan Howard, Patrick Ewing and Aaron McKie we...","Juwan Howard, Patrick Ewing and Aaron McKie we...",About a week before Michigan snatched the atte...,D,The Gray Eminences Guiding Ex-Stars Who Return...,"[Basketball (College), Coaches and Managers, G...",Sports
8,The authorities warned of potentially worse co...,The authorities warned of potentially worse co...,"NOWRA, Australia — The smoke is still smolderi...",A,"Despite Worsening Conditions, Australians Def...","[Australia, Wildfires, Evacuations and Evacuee...",Foreign
9,Ms. Warren’s fund-raising haul for the last th...,Ms. Warren’s fund-raising haul for the last th...,Senator Elizabeth Warren of Massachusetts rais...,A,Warren Says She Raised $21.2 Million In Quarter,"[Warren, Elizabeth, Campaign Finance, Presiden...",Politics


In [8]:
article_df.shape

(7585, 7)

In [10]:
article_df.drop

<bound method DataFrame.drop of                                                abstract  \
0     Attacks have been traditionally underreported,...   
1     He deftly manipulated electronic images to pro...   
2     The past definitely wasn’t past for directors ...   
3     Forget fad diets and fitness gimmicks. Just st...   
4     Linda Tracy and Peter Tracy had been married 3...   
...                                                 ...   
7580  The city could become one of the first in the ...   
7581  A New York City commission’s recommendations f...   
7582  Seth Meyers took a closer look at Dershowitz’s...   
7583  The secretary of state met with President Volo...   
7584  After an investigation revealed abuses at a sh...   

                                                snippet  \
0     Attacks have been traditionally underreported,...   
1     He deftly manipulated electronic images to pro...   
2     The past definitely wasn’t past for directors ...   
3     Forget fad diets 

In [11]:
article_df['abstract'][0]

'Attacks have been traditionally underreported, but fighting the scourge in hate crimes begins with better data, experts say.'

In [12]:
article_df.drop(columns = ['snippet', 'print_section','news_desk'], inplace= True, errors= 'ignore')
article_df.head()

,abstract,lead_paragraph,headline,keywords
0,"Attacks have been traditionally underreported,...",When slurs were spray-painted on a kosher mark...,Scrutiny Reveals a Rise in Anti-Semitic Crimes,"[ANTI-SEMITISM, Hate Crimes, New York City, Ch..."
1,He deftly manipulated electronic images to pro...,"Woody Vasulka, an experimental video artist wh...","Woody Vasulka, Who Foresaw Video as Art, Dies ...","[Vasulka, Woody, Deaths (Obituaries), Art, Kit..."
2,The past definitely wasn’t past for directors ...,"It’s Oscar time, which means everything new is...",The Pros and Cons of Nostalgia,"[Movies, ACADEMY AWARDS (OSCARS), 1917 (Movie)..."
3,Forget fad diets and fitness gimmicks. Just st...,"Daytime television talk shows, popular podcast...",None,"[Diet and Nutrition, Exercise, Weight, Cooking..."
4,Linda Tracy and Peter Tracy had been married 3...,When Donna Adelsberg ran into her best friend’...,"A Love So Persistent, They Had to Marry Twice","[Divorce, Separations and Annulments, Baltimor..."


In [13]:
article_df = article_df.dropna()
article_df

,abstract,lead_paragraph,headline,keywords
0,"Attacks have been traditionally underreported,...",When slurs were spray-painted on a kosher mark...,Scrutiny Reveals a Rise in Anti-Semitic Crimes,"[ANTI-SEMITISM, Hate Crimes, New York City, Ch..."
1,He deftly manipulated electronic images to pro...,"Woody Vasulka, an experimental video artist wh...","Woody Vasulka, Who Foresaw Video as Art, Dies ...","[Vasulka, Woody, Deaths (Obituaries), Art, Kit..."
2,The past definitely wasn’t past for directors ...,"It’s Oscar time, which means everything new is...",The Pros and Cons of Nostalgia,"[Movies, ACADEMY AWARDS (OSCARS), 1917 (Movie)..."
4,Linda Tracy and Peter Tracy had been married 3...,When Donna Adelsberg ran into her best friend’...,"A Love So Persistent, They Had to Marry Twice","[Divorce, Separations and Annulments, Baltimor..."
5,Split categories allow voters to spread the we...,What will happen at the Golden Globes? Part of...,A Crystal Ball For an Event Prone to Jolts,"[Movies, Golden Globes (Awards), Hustlers (Mov..."
...,...,...,...,...
7579,The narrow vote came after Republican senators...,WASHINGTON — The Senate brought President Trum...,"Senate Republicans Block Witnesses, 51 to 49, ...","[Presidential Election of 2020, United States ..."
7580,The city could become one of the first in the ...,New York City should consider allowing yellow ...,New York Is Urged to Consider Surge Pricing fo...,"[Taxicabs and Taxicab Drivers, Uber Technologi..."
7581,A New York City commission’s recommendations f...,"On a stretch of 11th Street in Park Slope, Bro...","The Push to Reform Property Tax Inequities, an...","[Property Taxes, Real Estate and Housing (Resi..."
7583,The secretary of state met with President Volo...,"KYIV, Ukraine — Secretary of State Mike Pompeo...","Amid Trump’s Trial, Pompeo Visits Kyiv and Vow...",[Trump-Ukraine Whistle-blower Complaint and Im...


### Removing punctuation using RegEx

In [14]:
# Removing punctuation
import re
def preprocessor(text):
    if not text:
        return ""
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

original_df = article_df.copy(deep=True)

article_df['abstract'] = article_df['abstract'].apply(preprocessor)
article_df['lead_paragraph'] = article_df['lead_paragraph'].apply(preprocessor)
article_df['headline'] = article_df['headline'].apply(preprocessor)

C:\Users\siddh\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\siddh\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\siddh\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

### Tokenizing documents using porter stemmer

In [15]:

from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.lower().split()


def tokenizer_porter(text):
    lst = [porter.stem(word) for word in text.split()]
    new_str = ""
    for i in lst:
        new_str+=i+' '
    return new_str[0:-1]

### I'm using an averaged perceptron tagger to extract the parts of speech from each record.

In [7]:
import nltk
nltk.download('averaged_perceptron_tagger')
text = tokenizer("And now for something completely different")

nltk.pos_tag(text)

AttributeError: type object 'scipy.io.matlab.mio5_utils.VarHeader5' has no attribute '__reduce_cython__'

### According to Turney, 2002 - only certain combinations of POS tags are to be used to derive sentiment. Mimicking this appraoch

In [8]:

all_types = ['LS', 'TO', 'VBN', "''", 'WP', 'UH', 'VBG', 'JJ', 'VBZ', '--', 'VBP', 'NN', 'DT', 'PRP', ':', 'WP$', 'NNPS', 'PRP$', 'WDT', '(', ')', '.', ',', '``', '$', 'RB', 'RBR', 'RBS', 'VBD', 'IN', 'FW', 'RP', 'JJR', 'JJS', 'PDT', 'MD', 'VB', 'WRB', 'NNP', 'EX', 'NNS', 'SYM', 'CC', 'CD', 'POS']
all_minus_nouns = ''
for i in all_types:
    if i != 'NN' and i!='NNS': 
        all_minus_nouns += i+'@'
all_minus_nouns = all_minus_nouns[:-1]
all_types_str = all_minus_nouns+'@NN@NNS'
patterns = [
            ['JJ', 'NN@NNS',all_types_str],
            ['RB@RBR@RBS','JJ',all_minus_nouns],
            ['JJ','JJ',all_minus_nouns],
            ['NN@NNS','JJ',all_minus_nouns],
            ['RB@RBR@RBS','VB@VBD@VBN@VBG', all_types_str]
            ]





In [18]:
text = tokenizer(article_df['abstract'][0])
tags = nltk.pos_tag(text)
tags

[('attacks', 'NNS'),
 ('have', 'VBP'),
 ('been', 'VBN'),
 ('traditionally', 'RB'),
 ('underreported', 'JJ'),
 ('but', 'CC'),
 ('fighting', 'VBG'),
 ('the', 'DT'),
 ('scourge', 'NN'),
 ('in', 'IN'),
 ('hate', 'JJ'),
 ('crimes', 'NNS'),
 ('begins', 'VBZ'),
 ('with', 'IN'),
 ('better', 'JJR'),
 ('data', 'NN'),
 ('experts', 'NNS'),
 ('say', 'VBP')]

In [11]:
def find_key_phrases(tags):
    phrases = []
    for i in range(len(tags)-3):
        curr_tags = tags[i:i+3]
        curr_types = [tag[1] for tag in curr_tags]
        for pattern in patterns:
            flag = all([z in p for z, p in zip(curr_types, pattern)])
            if flag:
                phrases.append(curr_tags[:2])
                break
    return phrases

### Creating a web scraper that searches bing.com using the search string "excellent AND [search term]" and "poor AND [search term]" and returns the number of hits for each. The log ratio of the excellent compared to the poor terms gives us an estimate of the sentiment of each phrase, and these sentiments are averaged all phrases in the article to give the final sentiment score

In [9]:
def results(word):
    url = "https://www.bing.com/search?q=%s"%word
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    res_text = soup.find('div',{'id':'b_tween'}).text
    num_res = int(res_text.split()[0].replace(',', ''))   
    return num_res

def bing_lookup(phrase):
    str_excellent = '"excellent"+AND+"%s"'%(phrase.replace(' ','+'))
    str_poor = '"poor"+AND+"%s"'%(phrase.replace(' ','+'))
    return results(str_excellent), results(str_poor)

### The words 'excellent' and 'poor' were chosen as they give a similar number of hits when searched by themselves

In [10]:
hits_poor = results('poor')
hits_excellent = results('excellent')
hits_poor, hits_excellent

(251000000, 241000000)

### For example, the phrase 'hate crimes' has far more hits with the phrase poor than with excellent, therefore it's likely to have a negative sentiment

In [22]:
hits_phrase_excellent, hits_phrase_poor = bing_lookup("hate crimes")

In [23]:
hits_phrase_excellent, hits_phrase_poor

(373000, 1160000)

In [24]:
def get_so(hits_phrase_excellent, hits_phrase_poor):
    return math.log((hits_phrase_excellent*hits_poor)/(hits_excellent*hits_phrase_poor), 2)

In [25]:
get_so(hits_phrase_excellent, hits_phrase_poor)

-1.58354776735357

In [26]:
# sentiments = []
# for phrase in phrases:
#     phrase_sentiments = []
#     for word1, word2 in phrase:
#         try:
#             hits_phrase_excellent, hits_phrase_poor = bing_lookup(word1[0]+' '+word2[0])
#         except:
#             continue
#         phrase_sentiments.append(get_so(hits_phrase_excellent, hits_phrase_poor))
    

In [27]:
phrases = []
for i in range(50):
    text = tokenizer(article_df['abstract'].iloc[i])
    tags = nltk.pos_tag(text)
    phrases.append(find_key_phrases(tags))
phrases[:5]


[[[('traditionally', 'RB'), ('underreported', 'JJ')],
  [('hate', 'JJ'), ('crimes', 'NNS')]],
 [[('deftly', 'RB'), ('manipulated', 'VBD')],
  [('electronic', 'JJ'), ('images', 'NNS')],
  [('jarring', 'JJ'), ('visions', 'NNS')],
  [('also', 'RB'), ('founded', 'VBD')]],
 [[('t', 'JJ'), ('past', 'NN')],
  [('martin', 'NN'), ('scorsese', 'JJ')],
  [('taika', 'JJ'), ('waititi', 'NN')],
  [('less', 'RBR'), ('palatable', 'JJ')]],
 [[('own', 'JJ'), ('weddings', 'NNS')]],
 [[('martin', 'NN'), ('scorsese', 'JJ')], [('bong', 'JJ'), ('joon', 'NN')]]]

In [28]:
sentiments = []
for phrase in phrases:
    phrase_sentiments = []
    for word1, word2 in phrase:
        try:
            hits_phrase_excellent, hits_phrase_poor = bing_lookup(word1[0]+' '+word2[0])
        except:
            continue
        phrase_sentiments.append(get_so(hits_phrase_excellent, hits_phrase_poor))
    sentiments.append(np.mean(phrase_sentiments))
sentiments[:5]

C:\Users\siddh\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\siddh\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[-3.452871462099974,
 -1.0386599298567223,
 0.18922674577197576,
 1.3984640218247029,
 -0.3545268428859249]

In [29]:
for i, sen in zip(article_df['abstract'].iloc[0:50], sentiments):
    print(i, sen, end="\n\n")
    
##next- plot correlation bw PMI-IR and nltk sentiment tagger- find outliers- test on pretagged dataset
# try lemmatization instead of stemming

attacks have been traditionally underreported but fighting the scourge in hate crimes begins with better data experts say  -3.452871462099974

he deftly manipulated electronic images to produce otherworldly sometimes jarring visions he also founded with his wife a landmark performance space  -1.0386599298567223

the past definitely wasn t past for directors like quentin tarantino martin scorsese and taika waititi each found a way to rewrite history though some versions were less palatable than others  0.18922674577197576

linda tracy and peter tracy had been married 37 years before divorcing in 2015 when their sons began planning their own weddings mom and dad rediscovered love  1.3984640218247029

split categories allow voters to spread the wealth but who prevails when quentin tarantino martin scorsese and bong joon ho compete  -0.3545268428859249

e s p conjuring the dead speaking with them shows like the thin place and our dear dead drug lord use the occult to put on a show  1.26209

In [30]:
for i in article_df['abstract'].iloc[0:5]:
    print(i, end="\n\n")

attacks have been traditionally underreported but fighting the scourge in hate crimes begins with better data experts say 

he deftly manipulated electronic images to produce otherworldly sometimes jarring visions he also founded with his wife a landmark performance space 

the past definitely wasn t past for directors like quentin tarantino martin scorsese and taika waititi each found a way to rewrite history though some versions were less palatable than others 

linda tracy and peter tracy had been married 37 years before divorcing in 2015 when their sons began planning their own weddings mom and dad rediscovered love 

split categories allow voters to spread the wealth but who prevails when quentin tarantino martin scorsese and bong joon ho compete 



In [33]:
# Comparing nltk VADER to PMi-IR classifier for first 50 examples
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
i=0
sid = SentimentIntensityAnalyzer()
for sentence in article_df.abstract.iloc[:50]:
    print(sentence)
    ss = sid.polarity_scores(sentence)
#     print(ss)
#     for k in sorted(ss):
#          print('{0}: {1}, '.format(k, ss[k]), end='')
    print('PMI-IR:', sentiments[i], 'VADER:',ss['compound'])
    i+=1
    print('\n')

attacks have been traditionally underreported but fighting the scourge in hate crimes begins with better data experts say 


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


NameError: name 'sentiments' is not defined

In [27]:

# Positive >= 0.5, Neutral b/w -0.5 and 0.5, Negative <=0.5
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews
# Convert to dataframe
documents = []
for fileid in movie_reviews.fileids():
    tag, filename = fileid.split('/')
    documents.append((tag, movie_reviews.raw(fileid)))
testing_df = pd.DataFrame(documents, columns=['target', 'document'])
testing_df.head()

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


,target,document
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [28]:
new_target = []
for i in testing_df.target:
    if i == 'neg':
        new_target.append(0)
    else:
        new_target.append(1)
testing_df['target'] = new_target

In [29]:
testing_df.tail()

,target,document
1995,1,wow ! what a movie . \nit's everything a movie...
1996,1,"richard gere can be a commanding actor , but h..."
1997,1,"glory--starring matthew broderick , denzel was..."
1998,1,steven spielberg's second epic film on world w...
1999,1,"truman ( "" true-man "" ) burbank is the perfect..."


In [34]:
vader_pred = []
vader = SentimentIntensityAnalyzer()
for sentence in testing_df['document']:
#     print(sentence)
    ss = vader.polarity_scores(sentence)
    if ss['compound'] > 0:
        vader_pred.append(1)
    else:
        vader_pred.append(0)
testing_df['vader'] = vader_pred
testing_df.head()

,target,document,vader
0,0,"plot : two teen couples go to a church party ,...",1
1,0,the happy bastard's quick movie review \ndamn ...,1
2,0,it is movies like these that make a jaded movi...,1
3,0,""" quest for camelot "" is warner bros . ' firs...",0
4,0,synopsis : a mentally unstable man undergoing ...,1


In [35]:
vader_accuracy = sum(testing_df['target'] == testing_df['vader'])/len(testing_df['target'])
vader_accuracy

0.6365

In [36]:
random_sel = testing_df.sample(50, random_state = 46)

In [37]:
# vader with preprocessing
random_sel['document'] == random_sel.document.apply(preprocessor).apply(tokenizer)

vader_pred = []
vader = SentimentIntensityAnalyzer()
for sentence in random_sel['document']:
#     print(sentence)
    ss = vader.polarity_scores(sentence)
    if ss['compound'] > 0:
        vader_pred.append(1)
    else:
        vader_pred.append(0)
random_sel['vader'] = vader_pred
# testing_df.head()
vader_accuracy = sum(random_sel['target'] == random_sel['vader'])/len(random_sel['target'])
vader_accuracy

0.54

In [38]:
# TODO: Run PMI-IR and compare
len(testing_df)

2000

In [39]:
testing_phrases = []
for i in range(len(random_sel)):
    text = tokenizer(random_sel.document.iloc[i])
    tags = nltk.pos_tag(text)
    testing_phrases.append(find_key_phrases(tags))
testing_phrases[:5]


[[[('high', 'JJ'), ('hopes', 'NNS')],
  [('over-the-top', 'JJ'), ('performance', 'NN')],
  [('tv', 'NN'), ('commercial', 'JJ')],
  [('goose', 'JJ'), ('bumps', 'NNS')],
  [('kooky', 'JJ'), ('mix', 'NN')],
  [('michael', 'JJ'), ('douglas', 'NNS')],
  [('dull', 'JJ'), ('kidnapping', 'NN')],
  [('douglas', 'JJ'), ('stars', 'NNS')],
  [('nathan', 'JJ'), ('conrad', 'NN')],
  [('new', 'JJ'), ('york', 'NN')],
  [('skye', 'JJ'), ('mccole', 'NN')],
  [('famke', 'JJ'), ('janssen', 'NN')],
  [('sean', 'JJ'), ("bean's", 'NN')],
  [('new', 'JJ'), ('patient', 'NN')],
  [('violent', 'JJ'), ('elisabeth', 'NN')],
  [('hidden', 'JJ'), ('bank', 'NN')],
  [('hard-nosed', 'JJ'), ('detective', 'JJ')],
  [('detective', 'JJ'), ('sandra', 'NN')],
  [('lo', 'JJ'), ('impression', 'NN')],
  [('nathan', 'JJ'), ('races', 'NNS')],
  [("elisabeth's", 'JJ'), ('head', 'NN')],
  [('too', 'RB'), ('much', 'JJ')],
  [('first', 'JJ'), ('part', 'NN')],
  [('only', 'JJ'), ('real', 'JJ')],
  [('real', 'JJ'), ('"', 'NN')],
  [('

In [40]:
bing_lookup('twenty forty')

(276000, 199000)

In [1]:
#previously done with np.mean(phrase_sentiments) instead of overall_sentiment 
testing_sentiments = []
for phrase in testing_phrases:
    phrase_sentiments = []
    for word1, word2 in phrase:
        try:
            hits_phrase_excellent, hits_phrase_poor = bing_lookup(word1[0]+' '+word2[0])
        except:
            continue
        phrase_sentiments.append(get_so(hits_phrase_excellent, hits_phrase_poor))
    overall_sentiment = 1 if sum([i > 0 for i in phrase_sentiments]) > sum([i < 0 for i in phrase_sentiments]) else 0
    testing_sentiments.append(overall_sentiment)
    print(phrase, overall_sentiment)
testing_sentiments[:5]

NameError: name 'testing_phrases' is not defined

In [43]:
with open('testing_sentiments.txt', 'w') as f:
    for item in testing_sentiments:
        f.write("%s\n" % item)

In [44]:
with open('testing_sentiments.txt') as f:
    lines = f.read().splitlines()

In [94]:
testing_sentiments_binary = []
for sent in testing_sentiments:
    if sent > 0:
        testing_sentiments_binary.append(1)
    else:
        testing_sentiments_binary.append(0)

In [107]:
random_sel['pmi_ir'] = testing_sentiments
random_sel['pmi_ir_binary'] = testing_sentiments_binary

In [112]:
random_sel.reset_index(inplace=True)
random_sel.to_csv('random_sel.csv')

In [113]:
random_sel.head()

,index,target,document,vader,pmi_ir,pmi_ir_binary
0,1893,1,let me open this one with a confession : i lov...,1,0.116205,1
1,898,0,"okay , okay . \nmaybe i wasn't in the mood to ...",0,-0.026904,0
2,1578,1,"for a movie about disco-era excess , "" 54 "" co...",1,0.228512,1
3,559,0,this is a film that i was inclined to like at ...,1,0.183806,1
4,88,0,the first film produced by adam sandler's happ...,1,0.674662,1


In [114]:
pmi_ir_accuracy = sum(random_sel['target'] == random_sel['pmi_ir_binary'])/len(random_sel['target'])
pmi_ir_accuracy

0.44

In [115]:
random_sel.loc[random_sel['target'] != random_sel['pmi_ir_binary']]

,index,target,document,vader,pmi_ir,pmi_ir_binary
3,559,0,this is a film that i was inclined to like at ...,1,0.183806,1
4,88,0,the first film produced by adam sandler's happ...,1,0.674662,1
8,707,0,one of the 90s' most unwelcome thriller trends...,0,0.525643,1
12,604,0,an affluent horse breeder's past comes up to h...,1,0.647974,1
14,1321,1,it must be some sort of warped critical nightm...,1,-0.073439,0
15,761,0,weighed down by tired plot lines and spielberg...,0,0.193470,1
16,611,0,it's a rare treat when a quality horror film i...,0,0.503908,1
17,1112,1,i'm not quite sure how best to go about writin...,0,-0.173417,0
18,576,0,so much for sweet returns . \nafter smart horr...,0,0.036388,1
19,487,0,adam sandler turns up the charm in his latest ...,1,0.610826,1
